In [1]:
from database_management import create_local_db,create_tables,upload_existing_data

In [2]:
import sqlite3
import pandas as pd

In [3]:
# !pip install -q transformers openai

In [4]:
# to create local db for first time
create_local_db()

db file created


In [5]:
'''to create tables - users, orders,categories
it deletes and recreates, if already exisiting'''
create_tables()

Tables created sucessfully


In [6]:
'''
I ve create some sample data on list of users, some already existing order details and a table of categories to which user can be categorised into
'''
upload_existing_data()

Data upload to tables completed successfuly


In [7]:
conn = sqlite3.connect("shopping.db")
cursor = conn.cursor()

In [8]:
cursor.execute("select user_name from users")
users_list = cursor.fetchall()

In [9]:
users_list = list(map(lambda x: x[0], users_list))
users_list

['Christy Zhu',
 'Julio Ruiz',
 'Curtis Lu',
 'Ruben Prasad',
 'Albert Alvarez',
 'Cole Watson',
 'Sydney Wright',
 'Colin Anand',
 'Rachael Martinez',
 'Emma Brown',
 'Courtney Edwards',
 'Edward Brown',
 'Brad Deng',
 'Martha Xu',
 'Katrina Raji',
 'Abigail Henderson',
 'Melanie Sanchez',
 'Louis Xie']

In [10]:
items_query = "select items from categories where items is not null"
cursor.execute(items_query)
all_items = cursor.fetchall()
all_items

[('books, pens, bags',),
 ('bags, shoes, mobiles, cloths, bikes, accessories',),
 ('bags, shoes, mobiles',),
 ('mobiles, cloths, accessories, electronics, makeups, shoes',),
 ('mobiles, cloths, accessories, shoes, bags, bikes',),
 ('mobiles, cloths, accessories, electronics, bikes, shoes',),
 ('mobiles, cloths, accessories, electronics, bikes, shoes, bags, books',),
 ('accessories, electronics, utencils, mobiles, cloths',),
 ('accessories, electronics, utencils, cloths, bags, books',)]

In [11]:
all_items = list(map(lambda x:x[0].split(', '), all_items))
# all_items

In [12]:
all_items = list({item for lst in all_items for item in lst})
all_items

['cloths',
 'books',
 'shoes',
 'utencils',
 'bags',
 'bikes',
 'accessories',
 'electronics',
 'mobiles',
 'makeups',
 'pens']

In [13]:
# !pip install -q together
import os
from dotenv import load_dotenv
load_dotenv('.env')
openai_endpoint = os.getenv("OPENAI_ENDPOINT")
openai_apikey = os.getenv("OPENAI_APIKEY")
deployment_name = os.getenv("DEPLOYMENT_NAME")

In [14]:
from openai import AzureOpenAI
client = AzureOpenAI(api_key = openai_apikey, azure_endpoint = openai_endpoint,api_version = "2024-05-01-preview")

In [15]:
response = client.chat.completions.create(model = deployment_name,
                               messages=[{"role":"user","content":"Hi"}]
                              )
response.choices[0].message.content

'Hello! How can I assist you today?'

In [16]:
cursor.execute("select category_name,category_id from categories")
out = cursor.fetchall()

In [17]:
catg_details = {key:value for key,value in out}
catg_details

{'student_low_buget': 1,
 'student_high_buget': 2,
 'student_avg_buget': 3,
 'female_earner': 4,
 'femaler_explorer': 5,
 'male_earner': 6,
 'male_explorer': 7,
 'familian_high_spender': 8,
 'familian_low_spender': 9,
 'mixed_category': 10}

In [18]:
def add_new_user(user_name:str,user_mail:str):
    query = """Insert into users (user_name,user_mail) 
    values (?,?)"""
    cursor.execute(query,(user_name,user_mail))
    conn.commit()
    print("user name added to db")
    cursor.execute("select user_name from users")
    users_list = cursor.fetchall()
    users_list = list(map(lambda x: x[0], users_list))

In [19]:
def recategorize_user(user_name:str,category:str):
    query = """update users
    set category_id =?
    where user_name = ?"""
    category_id = catg_details.get(category)
    cursor.execute(query,(category_id,user_name))
    conn.commit()
    # print("category id updated successfuly")

In [20]:
cursor.execute("select category_name,spend_capacity,items from categories")
category_details = cursor.fetchall()

In [21]:
def get_user_shoppig_details(user_name:str):
    query = """select o.order_name,o.order_value,c.category_name from orders o
    join users u on u.user_id=o.user_id
    join categories c on c.category_id=u.category_id
    where u.user_name=?"""
    cursor.execute(query,(user_name,))
    out = cursor.fetchall()
    return out

In [22]:
def add_to_shoppping_history(user_name:str,item:str,cost:int):
    query = """insert into orders (user_id,order_name,order_value)
    select user_id,?,? from users 
    where users.user_name==?"""
    cursor.evaluate(query,(item,cost,user_name))
    conn.commit()

In [23]:
def auto_recommend_next_item(user_name,next_item,cost):
    historic_data = get_user_shoppig_details(user_name)
    system_prompt = f"""You're a recomendation agent for user in a shopping app. 
    Currently few users are categorized into multiple categories based on thier historic shopping activites on type of items they had purchased along with their purchasing capacities.
    
    Now you have to recategorize the user based on thier next purchase activites and then recommend them with next item they would be intrested in buying."""
    user_input=f"""
    CATEGORY DETAILS : {category_details}
    
    USER HISTORIC DATA: {historic_data}
    
    NEW ITEM: {next_item}
    COST: {cost}
    """
    output_format="""{
    "NEXT_RECOMENDATION":<>
    "CATEGORY":<>
    }"""
    user_instructions="Make sure to return output without additional explainations, next recomendation should be just the item/s, category should be one among the exisitng list"
    response = client.chat.completions.create(model = deployment_name,
                               messages=[{"role":"system","content":system_prompt},
                                        {"role":"user","content":user_input},
                                        {"role":"user","content":user_instructions},
                                        {"role":"user","content":output_format}]
                              )
    out = response.choices[0].message.content
    import json
    out = json.loads(out)
    recategorize_user(user_name,out["CATEGORY"])
    return out

In [24]:
from pydantic import BaseModel
class recomendation(BaseModel):
    NEXT_RECOMENDATION:str
    CATEGORY:str

In [25]:
def start_shopping():
    print("Welcome, have a good shopping today :)")
    user_name = input("Please identify your self with your name")
    if user_name not in users_list:
        print("We see new to join, please sign up below.")
        mail = input("Please provide your mail-id")
        add_new_user(user_name,mail)
    print("you're set to shop now")
    is_proceed="yes"
    while is_proceed=="yes":
        new_item = input('Please provide item you wish to buy  ')  
        cost = input('Please provide cost here') 
        recomendations = auto_recommend_next_item(user_name,new_item,cost)           
        print(f"We have a new recommendation to you, intrested in buying more of this??? \n*********{recomendations["NEXT_RECOMENDATION"]}********* ")
        is_proceed = input("Wish to continue shopping? yes/no")
        

In [26]:
start_shopping()

Welcome, have a good shopping today :)


Please identify your self with your name Raju


We see new to join, please sign up below.


Please provide your mail-id raju@mail.com


user name added to db
you're set to shop now


...

Wish to continue shopping? yes/no yes
Please provide item you wish to buy   mobiles
Please provide cost here 1000


We have a new recommendation to you, intrested in buying more of this??? 
*********cloths, accessories********* 


Wish to continue shopping? yes/no yes
Please provide item you wish to buy   accessories
Please provide cost here 500


We have a new recommendation to you, intrested in buying more of this??? 
*********mobiles********* 


Wish to continue shopping? yes/no no
